In [16]:
# importing nessacry librares forour project
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
!pip install twilio
from twilio.rest import Client

data = {
    'temperature': [38, 35, 41, 37, 40, 42, 39],
    'humidity': [10, 12, 9, 11, 10, 8, 9],
    'wind_speed': [20, 22, 18, 21, 23, 19, 22],
    'heatwave': [1, 0, 1, 0, 1, 1, 1]
}

df = pd.DataFrame(data)

# Split data into features and target
X = df[['temperature', 'humidity', 'wind_speed']]
y = df['heatwave']  # Target (heatwave: 1 or 0)

# Split sets into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)

# Training the model
model.fit(X_train, y_train)

# Predict on test set
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

Model Accuracy: 100.00%


using open weather website to retireve real time data


In [31]:
import requests
import pandas as pd

# Function to get live weather data
def get_live_weather_data(city):
    API_KEY = "36bf03f00a17d9c44132b8c265f44a58"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"

    # Send request to OpenWeatherMap
    response = requests.get(url)

    if response.status_code == 200:
        weather_data = response.json()

        # Extracting our weather information
        temperature = weather_data['main']['temp']
        humidity = weather_data['main']['humidity']
        wind_speed = weather_data['wind']['speed']
        weather_description = weather_data['weather'][0]['description']

        # Return weather data as a dictionary
        return {
            "temperature": temperature,
            "humidity": humidity,
            "wind_speed": wind_speed,
            "weather_description": weather_description
        }
    elif response.status_code == 401:
        print("Unauthorized. Please check your API key.")
    else:
        print(f"Failed to retrieve data for {city}, error code: {response.status_code}")
        return None

live_weather_data = get_live_weather_data("jaipur")
print(live_weather_data)



{'temperature': 26.62, 'humidity': 61, 'wind_speed': 2.57, 'weather_description': 'haze'}


In [29]:
if live_weather_data:
    # Convert live weather data to DataFrame
    live_weather_df = pd.DataFrame([live_weather_data])

    # Selecting specific features for prediction
    live_weather_df = live_weather_df[['temperature', 'humidity', 'wind_speed']] # Select only the features used during training


    predicted_heat_wave = model.predict(live_weather_df)[0]

    # Set a low temperature threshold for testing(becoz its winter in india right now)
    TEMP_THRESHOLD = 23.2
    live_temperature = live_weather_data["temperature"]

    # Check if temperature exceeds the threshold
    if predicted_heat_wave == 1 or live_temperature > TEMP_THRESHOLD:
        print("Heatwave predicted or temperature exceeds threshold!")
    else:
        print("No heatwave predicted and temperature is within limits.")


Heatwave predicted or temperature exceeds threshold!


sending alert messages via twilio


In [36]:
!pip install twilio
from twilio.rest import Client


account_sid = 'ACbee5497388e89b372d0c638d5492be1f'
auth_token = '17cef9c1a280b4ce151bd2fbb3a0c8bd'
twilio_phone_number = '18645139962'
recipient_phone_number = '+919571797661'


client = Client(account_sid, auth_token)

# Function to send SMS
def send_sms_alert(message):
    message = client.messages.create(
        body=message,
        from_=twilio_phone_number,
        to=recipient_phone_number
    )
    print(f'SMS sent: {message.sid}')

  #mentioning our city
city="jaipur"

#condition for sending message
if predicted_heat_wave == 1 or live_temperature > TEMP_THRESHOLD:
    alert_message = f"Heatwave Alert: in {city}. Stay indoors and drink mint and raw mango juice!"
    send_sms_alert(alert_message)
    print("SMS alert sent!")
else:
    print("No SMS alert needed.")

SMS sent: SM5833fbdf7464bc79cb1552763a4aced4
SMS alert sent!
